In [485]:
#import packages
import torch
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import dia_matrix
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast
import scqubits as sc

In [486]:
#scqubit fixed params
omega_low = 1e-9 * 2 * np.pi  # Low frequency cutoff. Units: 2pi GHz
omega_high = 3 * 2 * np.pi  # High frequency cutoff. Units: 2pi GHz
t_exp = 1e4  #Measurement time. Units: ns
A_cc = 1e-7 # Critical current noise strength. Units of critical current I_c
A_flux = 1e-6   # Flux noise strength. Units: Phi_0
M= 400  # Mutual inductance between qubit and a flux line. Units: \Phi_0 / Ampere
R_0 = 50 # Characteristic impedance of a transmission line. Units: ohms
T = 0.015 # Typical temperature for a superconducting circuit experiment. Units: K

In [487]:
#General Noise Functions 

def process_op(native_op, energy_esys, truncated_dim):
        ##### transform operator from native basis to energy eigenbasis 
    return native_op


def calc_therm_ratio(
    omega: torch.Tensor
) -> torch.Tensor:
    return (sp.constants.hbar * omega) / (sp.constants.k * T)


def t1(
        noise_op: torch.Tensor,
        spectral_density: Callable,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
        EL: torch.Tensor
        
    ):
        # We assume that the energies in `evals` are given in the units of frequency
        # and *not* angular frequency. The function `spectral_density` is assumed to
        # take as a parameter an angular frequency, hence we have to convert.

        ground = eigvecs[:,0]
        excited = eigvecs[:,1]
        ground_E = eigvals[0]
        excited_E = eigvals[1]

        omega = 2 * np.pi * (excited_E - ground_E) * 1e9
        
        s = spectral_density(omega, EL) + spectral_density(-omega, EL) 
        ##Not sure if this is correct - ask xanthe re the comments i.         total:
        #if False return a time/rate associated with a transition from state i to state j.
        #if True return a time/rate associated with both i to j and j to i transitions
       
        rate = torch.matmul(noise_op.to(torch.complex128),ground.T.to(torch.complex128))
        rate = torch.matmul(excited.conj().to(torch.complex128),rate)
        rate = torch.pow(torch.abs(rate) , 2) * s

        return rate


def tphi(
        A_noise: float,
        noise_op: torch.Tensor,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
    ) -> float:

        ground = eigvecs[:,0]
        excited = eigvecs[:,1]
        ground_E = eigvals[0]
        excited_E = eigvals[1]

        rate = torch.abs(
            torch.matmul(ground.conj().to(torch.complex128),torch.matmul(noise_op.to(torch.complex128),ground.T.to(torch.complex128)))
            - torch.matmul(excited.conj().to(torch.complex128),torch.matmul(noise_op.to(torch.complex128),excited.T.to(torch.complex128)))
        )
  
        rate *=  A_noise * np.sqrt(2 * np.abs(np.log(omega_low * t_exp)))

        # We assume that the system energies are given in units of frequency and
        # not the angular frequency, hence we have to multiply by `2\pi`
        rate *= 2 * np.pi

        return rate


#flux_bias_line_spectral_density
def spectral_density_fbl(omega ,EL):
        therm_ratio = calc_therm_ratio(omega)
        s = (
            2
            * (2 * np.pi) ** 2
            * M**2
            * omega
            * sp.constants.hbar
            / complex(R_0).real
            * (1 / torch.tanh(0.5 * therm_ratio))
            / (1 + torch.exp(-therm_ratio))
            )
        s *= (2 * np.pi)  # We assume that system energies are given in units of frequency
        return s


#inductive_spectral_density
def q_ind_fun(omega):
    therm_ratio = abs(calc_therm_ratio(omega))
    therm_ratio_500MHz = calc_therm_ratio(
        torch.tensor(2 * np.pi * 500e6)
    )
    return (
        500e6
        * (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio_500MHz)
            * torch.sinh(1 / 2 * therm_ratio_500MHz)
            / torch.exp(1 / 2 * therm_ratio_500MHz)
        )
        / (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio)
            * torch.sinh(1 / 2 * therm_ratio)
            / torch.exp(1 / 2 * therm_ratio)
        )
    )

def spectral_density_ind(omega,EL):
    therm_ratio = calc_therm_ratio(omega)
    s = (
        2
        * EL
        / q_ind_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s

#T1 Effective (Over all noise channels)


#T2 Effective (Over all noise channels)

In [488]:
#Zeropi Specific Functions

def _cos_phi_operator(
    pt_count: int, 
    min_val: float, 
    max_val: float, 
    x: float):
    vals = np.cos(np.linspace(min_val, max_val, int(pt_count)) + x)
    
    cos_phi_matrix = torch.from_numpy(dia_matrix((vals, [0]), shape=(pt_count, pt_count)).toarray())
    
    return cos_phi_matrix

def _cos_theta_operator(ncut):
        dim_theta = 2 * ncut + 1
        cos_theta_matrix = torch.from_numpy(
            (
                0.5
                * (
                    sparse.dia_matrix(
                        ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                    )
                    + sparse.dia_matrix(
                        ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                    )
                )
            ).toarray()
        )
        return cos_theta_matrix


def _sin_phi_operator(
        pt_count: int, 
        min_val: float, 
        max_val:float,
        x: float):

        vals = np.sin(np.linspace(min_val, max_val, pt_count)+x)

        sin_phi_matrix = torch.from_numpy(
            sparse.dia_matrix(
            (vals, [0]), shape=(pt_count, pt_count)
            ).toarray()
        )
        return sin_phi_matrix
      
def _sin_theta_operator(ncut):
    dim_theta = 2 * ncut + 1
    sin_theta_matrix = torch.from_numpy(
            -0.5
            * 1j
            * (
                sparse.dia_matrix(
                    ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                )
                - sparse.dia_matrix(
                    ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                )
            ).toarray()
            )
    return sin_theta_matrix

      
def d_hamiltonian_d_EJ(flux , energy_esys, min_val, max_val, pt_count, ncut, truncated_dim):
    d_potential_d_EJ_mat = -2.0 * torch.kron(
            _cos_phi_operator(min_val=min_val, max_val=max_val, pt_count=pt_count, x=-2.0 * np.pi * flux / 2.0),
            _cos_theta_operator(ncut))
    return process_op(native_op=d_potential_d_EJ_mat, energy_esys=energy_esys, truncated_dim = truncated_dim)


def d_hamiltonian_d_flux(flux, ncut, EJ, dEJ, energy_esys, pt_count, max_val, min_val, truncated_dim):
    op_1 = torch.kron(
        _sin_phi_operator(pt_count =pt_count, max_val=max_val, min_val=min_val, x=-2.0 * np.pi * flux / 2.0),
        _cos_theta_operator(ncut)
        )
    op_2 = torch.kron(
            _cos_phi_operator(pt_count =pt_count, max_val=max_val, min_val=min_val, x=-2.0 * np.pi * flux / 2.0),
            _sin_theta_operator(ncut)
        )
    d_potential_d_flux_mat =  -2.0 * np.pi * EJ * op_1 - np.pi * EJ * dEJ * op_2
    
    return d_potential_d_flux_mat
    
    #process_op(native_op=d_potential_d_flux_mat, energy_esys=energy_esys, truncated_dim=truncated_dim)


def _phi_operator(
        pt_count: int, #values defined for grid
        min_val: float, 
        max_val:float, truncated_dim=truncated_dim):
 
    phi_matrix = sparse.dia_matrix((pt_count, pt_count))
    diag_elements = np.linspace(min_val, max_val, pt_count)
    phi_matrix.setdiag(diag_elements)

    return torch.from_numpy(phi_matrix.toarray())
                          

def _identity_theta(ncut):
    dim_theta = 2 * ncut + 1
    return torch.eye(dim_theta)


def phi_operator(energy_esys, min_val, max_val, pt_count, ncut, truncated_dim):
    native = torch.kron(_phi_operator(min_val=min_val, max_val=max_val, pt_count=pt_count),_identity_theta(ncut))
    return process_op(native_op=native, energy_esys=energy_esys, truncated_dim=truncated_dim)
    

In [489]:
#Computing T2

#Pramater Values
EJ = 0.25
EL = 10.0**(-2)
ECJ = 0.5
EC = None
ECS = 10.0**(-3)

#understand which paramaters we should vary?
dEJ = 0
dCJ = 0
ng = 0.1
flux = 0.23

#Fixed Values
ncut  = 30
truncated_dim = 10

#Grid Discretisation Paramaters 
pt_count = 10
max_val = 6*np.pi
min_val = -6*np.pi

phi_grid = sc.Grid1d(min_val=min_val, max_val=max_val, pt_count=pt_count)
zeropi = sc.ZeroPi(
            grid = phi_grid,
            EJ   = EJ,
            EL   = EL, 
            ECJ  = ECJ,
            EC   = EC,
            ECS  = ECS,
            ng   = ng,
            flux = flux,
            ncut = 30, 
            dEJ = dEJ, 
            dCJ = dCJ)

#Find H
H = torch.from_numpy(zeropi.hamiltonian().toarray())

In [490]:

#Solve H
eigvals,eigvecs = torch.linalg.eigh(H)
omega = 2 * np.pi * (eigvals[1]-eigvals[0]) * 1e9

energy_esys = 0

#Tphi 
tphi_1_over_f_cc = tphi(
    A_noise = A_cc, 
    noise_op= d_hamiltonian_d_EJ(flux=flux, energy_esys=energy_esys, min_val=min_val, max_val=max_val, pt_count=pt_count, ncut = ncut,truncated_dim=truncated_dim), 
    eigvals=eigvals, eigvecs=eigvecs
    )

tphi_1_over_f_flux=tphi(
    A_noise = A_flux, 
    noise_op= d_hamiltonian_d_flux(flux=flux, EJ=EJ, dEJ=dEJ, energy_esys=energy_esys, truncated_dim=truncated_dim,min_val=min_val, max_val=max_val, pt_count=pt_count, ncut=ncut ), 
    eigvals=eigvals, eigvecs=eigvecs
    )

#T1
t1_inductive = t1(
    noise_op=phi_operator(energy_esys, min_val=min_val, max_val=max_val, pt_count=pt_count, ncut=ncut, truncated_dim=truncated_dim), 
    spectral_density=spectral_density_ind, 
    eigvals=eigvals, eigvecs=eigvecs,  EL = EL)


t1_flux_bias_line = t1(
    noise_op=d_hamiltonian_d_flux(flux=flux, EJ=EJ, dEJ=dEJ, energy_esys=energy_esys, min_val=min_val, max_val=max_val, pt_count=pt_count,  ncut=ncut, truncated_dim=truncated_dim), 
    spectral_density=spectral_density_fbl, 
    eigvals=eigvals, eigvecs=eigvecs,  EL =None)


In [491]:
#compare my pytorch computations to scqubits

print(zeropi.t1_inductive() - t1_inductive.item())
print(zeropi.t1_flux_bias_line() - t1_flux_bias_line.item())
print(zeropi.tphi_1_over_f_cc() - tphi_1_over_f_cc.item())
print(zeropi.tphi_1_over_f_flux() - tphi_1_over_f_flux.item())

9.189771614756266e+41
5.800905263107999e+43
3963273.7311381036
1428239.3905558465


In [ ]:
#Create Hamiltonian For Zero Pi 


#Common
def first_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return

def second_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return



#Kinetic Part

def kinetic_matrix():

    identity_phi = torch.eye(pt_count)

    identity_theta = torch.eye(dim_theta)

    i_d_dphi_operator = torch.kron(first_derivative_matrix(prefactor=1j), identity_theta )

    kinetic_matrix_phi = second_derivative_matrix(prefactor =  -2.0*ECJ)

    ##
    diag_elements = 2.0 * ECS * torch.square(torch.arange(-ncut + ng, ncut + 1 + ng))
    kinetic_matrix_theta = torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    ##


    #
    diag_elements = torch.arange(-ncut, ncut + 1)
    n_theta_matrix= torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    #### native = sparse.kron(self._identity_phi(), n_theta_matrix, format="csc")
    ##### n_theta_operator = process_op(native_op=native, energy_esys=energy_esys)
    #

    kinetic_matrix = torch.kron(kinetic_matrix_phi, identity_theta)+ torch.kron(identity_phi, kinetic_matrix_theta)

    if dCJ != 0:
        kinetic_matrix -= (
                    2.0
                    * ECS
                    * dCJ
                    * i_d_dphi_operator()
                    * n_theta_operator()
        )
    return kinetic_matrix


#Potential Part 

def potential_matrix():

    grid_linspace = self.grid.make_linspace()
    phi_inductive_vals = self.EL * np.square(grid_linspace)
    phi_inductive_potential = sparse.dia_matrix(
            (phi_inductive_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()


    phi_cos_vals = np.cos(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_cos_potential = sparse.dia_matrix(
            (phi_cos_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    phi_sin_vals = np.sin(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_sin_potential = sparse.dia_matrix(
            (phi_sin_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    theta_cos_potential = (
            -self.EJ
            * (
                sparse.dia_matrix(
                    ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                )
                + sparse.dia_matrix(
                    ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                )
            )
        ).tocsc()

    potential_mat = (
            sparse.kron(phi_cos_potential, theta_cos_potential, format="csc")
            + sparse.kron(phi_inductive_potential, self._identity_theta(), format="csc")
            + 2
            * self.EJ
            * sparse.kron(self._identity_phi(), self._identity_theta(), format="csc")
        )
        
        
    if dEJ != 0:
        potential_mat += (
                EJ
                * dEJ
                * sparse.kron(phi_sin_potential, self._identity_theta(), format="csc")
                * self.sin_theta_operator()
            )
    return potential_mat